In [3]:
import pandas as pd
from requests import request
import json
from datetime import datetime
today_timestamp = datetime.now().strftime("%Y%m%d")

In [14]:
# Run this to download newest data

CURRENT_JOB = "seriea_results"
# Read credentials file
with open("credentials.json", mode="r") as creds_file:
    credentials = json.load(creds_file)

# Get response
url = credentials[CURRENT_JOB]["url"]
headers = credentials[CURRENT_JOB]["headers"]
response = request("GET", url, headers=headers)

# Read the data current data file
with open("data.json", mode="r") as saved_data_file:
    saved_data = json.load(saved_data_file)

# Save latest in archive directory
archive_path = f"archive/data_{today_timestamp}.json"
with open(archive_path, mode="w") as archive_file:
    json.dump(saved_data, archive_file)

# Check if response was successful
if response.status_code == 200:
    # Get the data from the API response
    temp_data = response.json()
    
    # Write the data to a JSON file
    file_path = f"downloads/raw_data_{today_timestamp}.json"
    with open(file_path, "w") as file:
        json.dump(temp_data, file)
    
    # Clean temp_data
    cleansed_temp_data = {}
    for matchday, game_list in temp_data[0].items():
        cleansed_temp_data[matchday.strip()] = game_list

    # Update records from the latest download        
    updated_data =  saved_data | cleansed_temp_data
    with open("data.json", mode="w") as updated_data_file:
        json.dump(updated_data, updated_data_file)
    
else:
    # Handle the error
    print('Failed to retrieve data from API. Error code:', response.status_code)

In [ ]:
# Run this block to see the results
with open("data.json", mode="r") as data_file:
    saved_data = json.load(data_file)

# need to figure out sorting this

for matchday, game_list in saved_data.items():
    print(matchday)
    for game in game_list:
        # print(game)
        print(f"{game['homeTeam']} {game['homeTeamScore']}-{game['awayTeamScore']} {game['awayTeam']}")
    print("<><><><><><>")